In [1]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
import torch.nn.functional as F

In [5]:
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

#model = AutoModelForMaskedLM.from_pretrained('roberta-base')
#tokenizer = AutoTokenizer.from_pretrained('roberta-base')

mask_id = tokenizer.mask_token_id
mask = tokenizer.mask_token

mask_id, mask

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(103, '[MASK]')

In [4]:
attraction_example = "The key to the cabinets " + mask + " over there."
#attraction_example = "The key to the cabinets" + (" " +mask)*3
tokenized_input = tokenizer(attraction_example, return_tensors='pt')
mask_location = list(tokenized_input['input_ids'][0]).index(mask_id)

In [5]:
tokenized_input

{'input_ids': tensor([[  101,  1996,  3145,  2000,  1996, 20053,   103,  2058,  2045,  1012,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
output = model(**tokenized_input)

In [7]:
logits = F.softmax(output.logits[:,mask_location,:], dim=1).squeeze()
logits[tokenizer.convert_tokens_to_ids('is')].item(), logits[tokenizer.convert_tokens_to_ids('are')].item()

(0.230303093791008, 0.001198835321702063)

In [8]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [9]:
attraction_example = "The key to the cabinets"
encoded_input = tokenizer(attraction_example, return_tensors='pt')
outputs = model(**encoded_input, labels=encoded_input["input_ids"])

In [10]:
encoded_input

{'input_ids': tensor([[  464,  1994,   284,   262, 43064]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [11]:
probs = F.softmax(outputs.logits[:,-1,:], dim=1).squeeze()

In [12]:
probs[tokenizer.convert_tokens_to_ids('Ġis')].item(), probs[tokenizer.convert_tokens_to_ids('Ġare')].item()

(0.34608548879623413, 0.05830276012420654)

In [ ]:
outputs = model(**encoded_input, labels=encoded_input["input_ids"], output_attentions=True)

In [ ]:
attentions = outputs[-1]